# Client API with Airflow

Start by running a local Airflow instance with some test DAGs:
```commandline
cd s2gos-airflow
pixi install
pixi run airflow standalone
```

Then run the S2GOS gateway server with the local Airflow instance (assuming
the local Airflow webserver runs on http://localhost:8080):

```commandline
pixi shell
export AIRFLOW_USERNAME admin
export AIRFLOW_PASSWORD ***********
s2gos-server run --service=s2gos_server.services.airflow.testing:service
```

Get the airflow user password from `s2gos-airflow/.airflow/simple_auth_manager_passwords.json.generated`.

In [19]:
from s2gos_client import Client
from s2gos_common.models import ProcessRequest

In [20]:
client = Client(server_url="http://127.0.0.1:8008")
client

In [21]:
client.get_capabilities()

Capabilities(title='Airflow Dev Service', description=None, links=[Link(href='http://127.0.0.1:8008/', rel='self', type='application/json', hreflang='en', title='get_capabilities'), Link(href='http://127.0.0.1:8008/openapi.json', rel='service', type='application/json', hreflang='en', title='openapi'), Link(href='http://127.0.0.1:8008/docs', rel='service', type='text/html', hreflang='en', title='swagger_ui_html'), Link(href='http://127.0.0.1:8008/docs/oauth2-redirect', rel='service', type='text/html', hreflang='en', title='swagger_ui_redirect'), Link(href='http://127.0.0.1:8008/redoc', rel='service', type='text/html', hreflang='en', title='redoc_html'), Link(href='http://127.0.0.1:8008/', rel='service', type='application/json', hreflang='en', title='get_capabilities'), Link(href='http://127.0.0.1:8008/conformance', rel='service', type='application/json', hreflang='en', title='get_conformance'), Link(href='http://127.0.0.1:8008/processes', rel='service', type='application/json', hreflang

In [22]:
client.get_conformance()

ConformanceDeclaration(conformsTo=['http://www.opengis.net/spec/ogcapi-processes-1/1.0/conf/core', 'http://www.opengis.net/spec/ogcapi-processes-1/1.0/conf/ogc-process-description', 'http://www.opengis.net/spec/ogcapi-processes-1/1.0/conf/json', 'http://www.opengis.net/spec/ogcapi-processes-1/1.0/conf/oas30', 'http://www.opengis.net/spec/ogcapi-processes-1/1.0/conf/job-list', 'http://www.opengis.net/spec/ogcapi-processes-1/1.0/conf/dismiss'])

In [23]:
client.get_processes()

ProcessList(processes=[ProcessSummary(title='Prime Processor', description='Returns the list of prime numbers between a `min_val` and `max_val`. ', keywords=None, metadata=None, additionalParameters=None, id='primes_between', version='0.0.0', jobControlOptions=None, outputTransmission=None, links=None), ProcessSummary(title='BaseModel Test', description=None, keywords=None, metadata=None, additionalParameters=None, id='return_base_model', version='0.0.0', jobControlOptions=None, outputTransmission=None, links=None), ProcessSummary(title='Generate scene for testing', description='Simulate a set scene images slices for testing. Creates an xarray dataset with `periodicity` time slices and writes it as Zarr into a temporary location. Requires installed `dask`, `xarray`, and `zarr` packages.', keywords=None, metadata=None, additionalParameters=None, id='simulate_scene', version='0.0.0', jobControlOptions=None, outputTransmission=None, links=None), ProcessSummary(title='Sleep Processor', des

In [24]:
client.get_process(process_id="sleep_a_while")

ProcessDescription(title='Sleep Processor', description='Sleeps for `duration` seconds. Fails on purpose if `fail` is `True`. Returns the effective amount of sleep in seconds.', keywords=None, metadata=None, additionalParameters=None, id='sleep_a_while', version='0.0.0', jobControlOptions=None, outputTransmission=None, links=None, inputs={}, outputs={})

In [27]:
client.get_jobs()

JobList(jobs=[JobInfo(processID='sleep_a_while', type=<JobType.process: 'process'>, jobID='sleep_a_while__20250724103654_1', status=<JobStatus.accepted: 'accepted'>, message=None, created=datetime.datetime(2025, 7, 24, 10, 36, 54, 547216, tzinfo=TzInfo(UTC)), started=None, finished=None, updated=None, progress=None, links=None, traceback=None)], links=[Link(href='http://127.0.0.1:8008/jobs', rel='self', type='application/json', hreflang='en', title='get_jobs')])

In [29]:
client.execute_process(process_id="primes_between", request=ProcessRequest())

JobInfo(processID='primes_between', type=<JobType.process: 'process'>, jobID='primes_between__20250724103758_3', status=<JobStatus.accepted: 'accepted'>, message=None, created=datetime.datetime(2025, 7, 24, 10, 37, 58, 263463, tzinfo=TzInfo(UTC)), started=None, finished=None, updated=None, progress=None, links=None, traceback=None)

In [30]:
client.get_jobs()

JobList(jobs=[JobInfo(processID='primes_between', type=<JobType.process: 'process'>, jobID='primes_between__20250724103758_3', status=<JobStatus.accepted: 'accepted'>, message=None, created=datetime.datetime(2025, 7, 24, 10, 37, 58, 263463, tzinfo=TzInfo(UTC)), started=None, finished=None, updated=None, progress=None, links=None, traceback=None), JobInfo(processID='sleep_a_while', type=<JobType.process: 'process'>, jobID='sleep_a_while__20250724103654_1', status=<JobStatus.accepted: 'accepted'>, message=None, created=datetime.datetime(2025, 7, 24, 10, 36, 54, 547216, tzinfo=TzInfo(UTC)), started=None, finished=None, updated=None, progress=None, links=None, traceback=None)], links=[Link(href='http://127.0.0.1:8008/jobs', rel='self', type='application/json', hreflang='en', title='get_jobs')])

In [ ]:
# client.get_job_results("job_1")

In [ ]:
for job in client.get_jobs().jobs:
    client.dismiss_job(job.jobID)

In [32]:
client.get_jobs()

JobList(jobs=[JobInfo(processID='primes_between', type=<JobType.process: 'process'>, jobID='primes_between__20250724103758_3', status=<JobStatus.accepted: 'accepted'>, message=None, created=datetime.datetime(2025, 7, 24, 10, 37, 58, 263463, tzinfo=TzInfo(UTC)), started=None, finished=None, updated=None, progress=None, links=None, traceback=None), JobInfo(processID='sleep_a_while', type=<JobType.process: 'process'>, jobID='sleep_a_while__20250724103654_1', status=<JobStatus.accepted: 'accepted'>, message=None, created=datetime.datetime(2025, 7, 24, 10, 36, 54, 547216, tzinfo=TzInfo(UTC)), started=None, finished=None, updated=None, progress=None, links=None, traceback=None)], links=[Link(href='http://127.0.0.1:8008/jobs', rel='self', type='application/json', hreflang='en', title='get_jobs')])

In [33]:
client.get_job("primes_between__20250724103758_3")

JobInfo(processID='primes_between', type=<JobType.process: 'process'>, jobID='primes_between__20250724103758_3', status=<JobStatus.accepted: 'accepted'>, message=None, created=datetime.datetime(2025, 7, 24, 10, 37, 58, 263463, tzinfo=TzInfo(UTC)), started=None, finished=None, updated=None, progress=None, links=None, traceback=None)